In [1]:
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
import nltk
import re
import pandas as pd
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from itertools import islice
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

In [14]:
def text_process(text):
    '''
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Return the cleaned text as a list of words
    4. Remove words
    '''
    stemmer = WordNetLemmatizer()
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join([i for i in nopunc if not i.isdigit()])
    nopunc = [word.lower() for word in nopunc.split()
              if word not in stopwords.words('english')]
    return [stemmer.lemmatize(word) for word in nopunc]

train_df = pd.read_csv("cleaned.csv")
train_comments = train_df['cleaned_text']

vectorizer = TfidfVectorizer(analyzer=text_process,ngram_range=(1,3))
tfidfconvert = vectorizer.fit(train_comments)
X_transformed = tfidfconvert.transform(train_comments)

len(tfidfconvert.vocabulary_)

from sklearn.cluster import KMeans
modelkmeans = KMeans(n_clusters=60, init='k-means++', n_init=100)
modelkmeans.fit(X_transformed)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=60, n_init=100, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [17]:
print('Clustered distribution of labels:\n', pd.DataFrame(modelkmeans.labels_)[0].value_counts())

Clustered distribution of labels:
 24    2188
20    1209
37     361
5      331
6      264
54     258
55     255
43     248
45     235
4      231
32     228
30     217
12     212
21     205
17     204
42     199
3      197
52     196
33     195
50     190
18     182
10     181
9      179
15     178
47     174
49     173
13     173
25     172
8      170
38     167
14     165
36     154
41     154
2      153
1      153
29     148
40     140
0      138
46     135
16     134
11     131
22     129
27     124
59     120
35     107
31     104
34     103
7      101
57      95
48      88
39      86
26      84
53      82
58      79
28      75
56      53
44      50
19      49
51      19
23      17
Name: 0, dtype: int64
